In [4]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'../../modules')

In [5]:
import numpy as np
import factors
import exact_inference

# Sampling top down
When sampling a pgm one option is to make the full joint and sample from that. This requires making one large factor which might not be viable. Another option is to sample the top (root) nodes first, then condition all further samples on their parent nodes. This means you can sample the full joint without needing to merge all factors. <br>
Example: Making the graph $A$ and $B$ (independent) cause $C$ which then causes $D$ and $E$ (independent given $C$)

In [6]:
# making random values
A = factors.Factor(["A"],[2])
B = factors.Factor(["B"],[2])
C = factors.Factor(["C","A","B"],[2,2,2])
D = factors.Factor(["D","C"],[2,2])
E = factors.Factor(["E","C"],[2,2])
all_factors = []
for f in [A,B,C,D,E]:
    f.set_all(np.random.rand(np.prod(f.array.shape)))
    cond_f = factors.condition(f,list(f.names[1:]))
    all_factors.append(cond_f)
for f in all_factors:
    print(f)

A  Values (10 dp)
0  0.6101553093
1  0.3898446907

B  Values (10 dp)
0  0.853821384
1  0.146178616

C  A  B  Values (10 dp)
0  0  0  0.661781136
0  0  1  0.4906691389
0  1  0  0.561660912
0  1  1  0.742833305
1  0  0  0.338218864
1  0  1  0.5093308611
1  1  0  0.438339088
1  1  1  0.257166695

D  C  Values (10 dp)
0  0  0.2071755147
0  1  0.62738997
1  0  0.7928244853
1  1  0.37261003

E  C  Values (10 dp)
0  0  0.7921225093
0  1  0.958033264
1  0  0.2078774907
1  1  0.041966736



In [10]:
# Samples by setting the variables in order from top to bottom. Requires a directed factor graph. This is probably not going to work for an undirected graph.
def joint_sample_top_down(all_factors):
    assigned_variable_names = []
    variable_assignments = []
    remaining_factors = all_factors.copy()
    
    while(len(remaining_factors)>0): #  while variables haven't been assigned
        new_assigned_variable_names = assigned_variable_names.copy()
        new_variable_assignments = variable_assignments.copy()
        new_remaining_factors = []
        for f in remaining_factors:
            # grab variables with just themselves in the factor, and every variable which has all parent values set.
            if(len(f.names)==1 or np.prod([i in assigned_variable_names for i in f.names[1:]])==1):
                conditioned_factor = factors.drop_variables(f,assigned_variable_names,variable_assignments)
                new_variable_assignments.append(factors.sample(conditioned_factor,1)[0][0]) # sample a new value
                new_assigned_variable_names.append(f.names[0])
            else:
                new_remaining_factors.append(f)
        assigned_variable_names = new_assigned_variable_names
        variable_assignments = new_variable_assignments
        remaining_factors = new_remaining_factors
    return assigned_variable_names,variable_assignments

**Checking the results:**

In [11]:
all_samples = []
for n in range(5000):
    names,assignments = joint_sample_top_down(all_factors)
    all_samples.append(assignments)
all_samples = np.array(all_samples)
prob_C_sampled = factors.Factor(["C"],[2])
prob_C_sampled.set([0],np.sum(all_samples[:,2]==0)/all_samples.shape[0])
prob_C_sampled.set([1],np.sum(all_samples[:,2]==1)/all_samples.shape[0])
print(prob_C_sampled)

C  Values (10 dp)
0  0.6244
1  0.3756



In [12]:
prob_C_exact = exact_inference.sum_product_variable_elimination(all_factors,[],[],["A","B","D","E"])
print(prob_C_exact)

C  Values (10 dp)
0  0.6178124988
1  0.3821875012



# Direct Sampling, Likelihood weighted sampling, Gibbs Sampling
One problem with the above is that you can only sample the full joint this way. But often we need to sample a conditional distribution instead. If it is difficult to sample if the observed variables are the bottom nodes in a pgm, as it means sampling all parent nodes backwards. Whereas if the root nodes are known then it is easy, as above. There are a few options to deal with this.

### Direct Sampling
With direct sampling you sample the full joint like above and simply throw away all samples which don't match the observed variables. This is basically rejection sampling. Say we want to know $P(C)$ as above, but only if $A$ is 0 (using the same samples as before)

In [13]:
samples_where_A0 = all_samples[all_samples[:,0]==0]
prob_C_given_A_sampled = factors.Factor(["C"],[2])
prob_C_given_A_sampled.set([0],np.sum(samples_where_A0[:,2]==0)/samples_where_A0.shape[0])
prob_C_given_A_sampled.set([1],np.sum(samples_where_A0[:,2]==1)/samples_where_A0.shape[0])
print(prob_C_given_A_sampled)

C  Values (10 dp)
0  0.6388979993
1  0.3611020007



**checking the approximation is close to the exact value**

In [14]:
prob_C_given_A_exact = exact_inference.sum_product_variable_elimination(all_factors,["A"],[0],["B","D","E"])
print(prob_C_given_A_exact)

C  Values (10 dp)
0  0.6367682211
1  0.3632317789



### Likelihood weighting
With Likelihood weighting you sample all unknown variables as normal, but set the known variables to their value. This means all children are correctly sampled based on the parent values, but the parents are not sampled based on the children. So, you are sampling correct looking variables, but with a slightly incorect distribution. This is Importance Sampling applied to factors.
$$P(X=x_n) = \int \mathbb{1}(x=x_n)p(x) dx \approx \frac{1}{N} \sum_{i=1}^N \mathbb{1}(x_i=x_n) $$
Becomes:
$$\int \mathbb{1}(x=x_n)p(x) dx \approx \frac{1}{\sum \frac{p(x_n)}{q(x_n)}} \sum_{i=1}^N \frac{\mathbb{1}(x_i=x_n)p(x_n)}{q(x_n)} $$
Using the formula for normalized importance sampling. The distribution we sample from, $q(x)$, is constructed as above by setting observed values. The true probability $p(x)$ is the multiplication of all of the normalized conditional probabilities regardless of whether the observed value was set or not. $q(x)$ is the same, but $1$ whereever the value was assigned by evidence. Therefore, $\frac{p(x)}{q(x)}$ (the weight) is just the product of the probabilities at the observed variables.

In [15]:
# Likelihood weighted sampling. For normalized importance sampling to pgms. 
# Similar to above joint method but sets all observed variables and returns weight.
def likelihood_weighting_top_down(all_factors,known_vars,evidence):
    assigned_variable_names = []
    variable_assignments = []
    weight = 1
    remaining_factors = all_factors.copy()
    
    while(len(remaining_factors)>0):
        new_assigned_variable_names = assigned_variable_names.copy()
        new_variable_assignments = variable_assignments.copy()
        new_remaining_factors = []
        for f in remaining_factors:
            if(len(f.names)==1 or np.prod([i in assigned_variable_names for i in f.names[1:]])==1):
                var_dropped_factor = factors.drop_variables(f,assigned_variable_names,variable_assignments)
                conditioned_factor = factors.condition(var_dropped_factor)
                if(f.names[0] in known_vars):
                    evid = evidence[known_vars.index(f.names[0])]
                    new_variable_assignments.append(evid)
                    weight *= conditioned_factor.get([evid])
                else:
                    sample = factors.sample(conditioned_factor,1)[0][0]
                    new_variable_assignments.append(sample)
                    
                new_assigned_variable_names.append(f.names[0])
            else:
                new_remaining_factors.append(f)
        assigned_variable_names = new_assigned_variable_names
        variable_assignments = new_variable_assignments
        remaining_factors = new_remaining_factors
    return assigned_variable_names,variable_assignments,weight

**Checking results again:**

In [16]:
all_LW_samples = []
all_LW_weights = []
for n in range(1000):
    names,assignments,weight = likelihood_weighting_top_down(all_factors,["A"],[0])
    all_LW_samples.append(assignments)
    all_LW_weights.append(weight)
all_LW_samples = np.array(all_LW_samples)
all_LW_weights = np.array(all_LW_weights)

prob_C_given_A_LW_sampled = factors.Factor(["C"],[2])
prob_C_given_A_LW_sampled.set([0],np.sum(all_LW_weights*(all_LW_samples[:,2]==0))/np.sum(all_LW_weights))
prob_C_given_A_LW_sampled.set([1],np.sum(all_LW_weights*(all_LW_samples[:,2]==1))/np.sum(all_LW_weights))
print(prob_C_given_A_LW_sampled)

C  Values (10 dp)
0  0.639
1  0.361



### Gibbs sampling
An alternative was to do inference is to generate samples of the conditional distribution using gibbs sampling. Gibbs starts with a random set of variable values and proceedes by conditioning each variable on all variables within its markov blanket. This makes it tractable. It is a variant of the Markov Chain Monte Carlo (MCMC) technique for generating samples. 

In [29]:
# Gibbs sampling. Done with a step method and a method to generate lots of samples. 
# Quite complicated, but mainly because of indexing issues. I pass the markov blanket factors so they don't need to be recalculated.
# The markov blankets are just the factor product of all variables connected to a node. The gibbs step updates a sample in place.
def gibbs_step(all_variable_markov_blankets,fixed_variables,all_variable_names,current_state_values):
    for var_name in all_variable_names:
        if(not var_name in fixed_variables):
            joint = all_variable_markov_blankets[all_variable_names.index(var_name)]
            index = all_variable_names.index(var_name)
            other_var_names = list(all_variable_names[:index])+list(all_variable_names[(index+1):])
            other_var_vals = list(current_state_values[:index])+list(current_state_values[(index+1):])
            slc = [slice(None)]*len(joint.names)
            for i in range(len(all_variable_names)):
                if(all_variable_names[i] in joint.names):
                    slc[joint.names.index(all_variable_names[i])]=slice(current_state_values[i],current_state_values[i]+1)
            joint_index = joint.names.index(var_name)
            slc[joint_index]=slice(0,joint.array.shape[joint_index])
            array_slice = np.squeeze(joint.array[tuple(slc)])
            norm_array_slice = array_slice/np.sum(array_slice)
            cond_joint = factors.drop_variables(joint,other_var_names,other_var_vals)
            sample = np.random.choice(np.arange(joint.array.shape[joint_index]),1,p=norm_array_slice)
            #print(sample)
            current_state_values[index]=sample
    return current_state_values

In [30]:
# Gibbs sampling begins by making a random vector of values and then applies the gibbs step repeatedly. 
def gibbs_sampling(all_factors,known_vars,evidence,N):
    all_names = []
    # All this below is just to make a general random vector for a factor (and set the evidence)
    for f in all_factors:
        all_names+=list(f.names)
    all_names = list(np.unique(all_names))
    current_state = np.zeros(len(all_names)).astype(int) # start with 0's
    for i,name in enumerate(all_names):
        for f in all_factors: # find a factor to get the sample
            if(name in f.names):
                shape = f.array.shape[list(f.names).index(name)]
                current_state[i]=np.random.randint(0,shape)
                break
    for i in range(len(known_vars)):
        current_state[all_names.index(known_vars[i])]=evidence[i]
    
    # This is for making the markov blanket factors
    all_variable_markov_blankets = []
    for var_name in all_names:
        markov_blanket = []
        for f in all_factors:
            if(var_name in f.names):
                markov_blanket.append(f)
        all_variable_markov_blankets.append(factors.multiple_factor_product(markov_blanket))
    
    # This is the core loop
    all_visited_states = []
    for n in range(N):
        current_state = gibbs_step(all_variable_markov_blankets,known_vars,all_names,current_state)
        all_visited_states.append(current_state.copy())
    return np.array(all_visited_states)

In [32]:
samples = gibbs_sampling(all_factors,["A"],[0],10000)

In [33]:
average_C_given_A = np.mean(samples[100::10]==0,axis=0)[2]
print(average_C_given_A)

0.6161616161616161


As expected, this value matches the exact value closely

In [34]:
print(exact_inference.sum_product_variable_elimination(all_factors,["A"],[0],["B","D","E"]))

C  Values (10 dp)
0  0.6367682211
1  0.3632317789



The main sections of code here can be found in the pgm_sampling module

In [35]:
import pgm_sampling #gibbs_sampling,likelihood_weighting_top_down,joint_sample_top_down

In [36]:
gibbs_samples = pgm_sampling.gibbs_sampling(all_factors,["C"],[1],10)
print(gibbs_samples)

[[1 0 1 1 0]
 [0 0 1 1 0]
 [1 0 1 1 0]
 [0 0 1 1 0]
 [0 0 1 1 0]
 [1 0 1 0 0]
 [1 1 1 0 0]
 [0 0 1 0 0]
 [0 0 1 1 0]
 [0 0 1 0 0]]


In [41]:
_,likelihood_sample,weight = pgm_sampling.likelihood_weighting_top_down(all_factors,["C"],[1])
print(likelihood_sample)

[1, 1, 1, 0, 0]


In [43]:
_,joint_sample = pgm_sampling.joint_sample_top_down(all_factors)
print(joint_sample)

[0, 1, 0, 1, 0]
